## LangChain with Simple Chain Pattern
> **Expected Output**: Action:

This code is setting up a LangChain example that connects to a custom OpenAI-compatible large language model (LLM) endpoint using credentials from the environment, then prompts it to suggest a provide a 5 lines of overview of "VMware Tanzu Hub"


In [7]:
import os
import requests
import json
from openai import OpenAI
import httpx
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
import warnings
from cfenv import AppEnv
import sys, os
from tanzu_utils import CFGenAIService
warnings.filterwarnings('ignore')

In [8]:
# Load CF environment
env = AppEnv()

# load your service details replace name with your Gen AI service.  Gen AI service is bound to the app
chat_service = CFGenAIService("tanzu-gpt-oss-120b")

# List available models
models = chat_service.list_models()
for m in models:
    print(f"- {m['name']} (capabilities: {', '.join(m['capabilities'])})")

# construct chat_credentials
chat_credentials = {
    "api_base": chat_service.api_base + "/openai/v1",
    "api_key": chat_service.api_key,
    "model_name": models[0]["name"]
}

- openai/gpt-oss-120b (capabilities: CHAT, TOOLS)


In [9]:
# 2. HTTP client (optional but recommended for custom config)
httpx_client = httpx.Client(verify=False)  # verify=False if your endpoint needs --insecure

# 3. Initialize the LLM
llm = ChatOpenAI(
    temperature=0.9,
    model=chat_credentials["model_name"],   # model name from CF service
    base_url=chat_credentials["api_base"],  # OpenAI-compatible endpoint
    api_key=chat_credentials["api_key"],    # Bearer token
    http_client=httpx_client
)

In [10]:
# Create a prompt template
prompt = ChatPromptTemplate.from_template(
    "Please provide a 5 lines of overview of {topic}, including its key features, components, and integrations within the VMware Tanzu ecosystem."
)

# Create the chain
chain = LLMChain(llm=llm, prompt=prompt)

# Define the topic
topic = "VMware Tanzu Hub"

# Run the chain
response = chain.run(topic)
print(response)

**VMware Tanzu Hub – 5‑line overview**

1. **Unified Control Plane** – Tanzu Hub serves as a central, cloud‑native console that aggregates visibility, policy, and lifecycle management for all Tanzu‑managed Kubernetes clusters across on‑prem, public‑cloud, and edge environments.  
2. **Key Features** – Multi‑cluster inventory, automated compliance checks, version‑drift detection, integrated cost‑and‑usage analytics, and role‑based access control (RBAC) that propagate across the fleet.  
3. **Core Components** – Tanzu Hub UI/API, the Tanzu Mission Control service (for cluster provisioning & governance), and the Tanzu Observability & Logging back‑ends that feed telemetry into the hub.  
4. **Ecosystem Integrations** – Seamlessly connects with Tanzu Kubernetes Grid (TKG), Tanzu Application Service, Tanzu Data Services, and Tanzu Build Service, enabling a single pane of glass for workloads, data platforms, and CI/CD pipelines.  
5. **Extensibility & Automation** – Provides a programmable SD